## Data cleaning (data source 1)

**This code cleans up the tweets that come from data set no. 1, and prepares them to be merged to data set no. 2**  

**This code will not run, as the raw data is not contained in the repository. The files were too large to host on Github; we are hosting only the cleaned data**

In [49]:
import pandas as pd

In [50]:
data_1 = pd.read_csv('raw_data/DatasetOne.csv')

In [51]:
# Drop columns with only nulls
null_col_bool = [i for i in [data_1.isna().sum() != len(data_1)][0]]
cols_keep = data_1.columns[null_col_bool]
data_1 = data_1.loc[:, cols_keep]

In [52]:
# Drop this columns because all values are zero (these are related to Instagram and Linkedin attributes, and we ont have Instagram or Linknedin data)
cols_drop = [col for col in data_1.select_dtypes(include=['int', 'float']) if data_1[col].sum() == 0]
data_1.drop(columns = cols_drop, inplace=True)

In [53]:
# Drop columns because all entries are 0
0 == data_1['Is Syndicated'].sum() == data_1['Linkedin Sponsored'].sum() == data_1['Starred'].sum() == data_1['Checked'].sum()

# Drop columns becuase they only have one unique value (True for example)
1 == data_1['Total Monthly Visitors'].nunique() == data_1['Content Source'].nunique() == data_1['Content Source Name'].nunique() == data_1['Page Type Name'].nunique() == data_1['Pub Type'].nunique()

True

In [54]:
# Non useful columns 
non_useful_cols = ['Avatar', 'Latitude', 'Longitude', 'Total Monthly Visitors', 'Content Source', 'Is Syndicated', 'Linkedin Sponsored', 'Starred', 'Content Source Name', 'Page Type Name', 'Pub Type', 'Checked']
data_1.drop(columns = non_useful_cols, inplace=True)

In [55]:
data_2 = pd.read_csv('raw_data/DatasetTwo.csv')
data_2 = data_2.loc[:, cols_keep]
data_2.drop(columns = cols_drop, inplace=True)
data_2.drop(columns = non_useful_cols, inplace=True)

In [56]:
# tweets from data_1 and data_2
data_3 = pd.concat([data_2, data_1])

In [57]:
len(data_3), data_3.Title.duplicated().sum(), data_3['Query Id'].duplicated().sum()

(814103, 753791, 814102)

In [58]:
data_3.columns

Index(['Query Id', 'Query Name', 'Date', 'Title', 'Url', 'Domain', 'Sentiment',
       'Page Type', 'Language', 'Country Code', 'Continent Code', 'Continent',
       'Country', 'City Code', 'Account Type', 'Added', 'Author', 'City',
       'Entity Info', 'Expanded URLs', 'Full Name', 'Full Text', 'Gender',
       'Hashtags', 'Impact', 'Impressions', 'Interest', 'Location Name',
       'Media URLs', 'Mentioned Authors', 'Original Url', 'Professions',
       'Resource Id', 'Short URLs', 'Thread Author', 'Thread Created Date',
       'Thread Entry Type', 'Thread Id', 'Twitter Author ID',
       'Twitter Followers', 'Twitter Following', 'Twitter Reply Count',
       'Twitter Reply to', 'Twitter Retweet of', 'Twitter Retweets',
       'Twitter Likes', 'Twitter Tweets', 'Twitter Verified', 'Updated',
       'Reach (new)', 'Engagement Type', 'Region', 'Region Code',
       'Weblog Title'],
      dtype='object')

In [59]:
data_3.drop_duplicates(subset = ['Title'], inplace=True)

In [60]:
len(data_3)

60312

In [61]:
import numpy as np

**Classificaiton of accounts into media, government and public was an interative and slightly manual process**

In [62]:
potential_news_org = [i for i in data_3['Author'][(data_3['Account Type'] == 'organisational') & (data_3['Twitter Verified'] == True)].value_counts().index]

In [63]:
individual_verified = [i for i in data_3['Author'][(data_3['Account Type'] == 'individual') & (data_3['Twitter Verified'] == True)].value_counts().index]

In [64]:
# USing regular expression to classify the greatest number of tweeter handles into media accounts
import re
count = 0
news_org_list = []
check_by_hand = []
pattern = re.compile(r'news|abc|cnn|cbs|reuters|AP|times|fox|post|nbc|cnbc|bloomberg|gazette|hours|politico|cspan|politics|msn|journal|Tribune|daily|wsj|yahoo|tv|sun|time|radio|chronicle|herald|today', re.IGNORECASE) 
for item in potential_news_org:
    if pattern.search(item):
        news_org_list.append(item)
    else:
        check_by_hand.append(item)

In [65]:
# MK NEWS ACCOUNTS
mk_news_org = pd.read_csv('raw_data/news_accounts_dataset3.csv')
mk_news_org = mk_news_org.SENDER.to_list()

In [66]:
news_orgs_list2 = []
check_by_hand_2 = []
for account in check_by_hand:
    if account in mk_news_org:
      news_orgs_list2.append(account)
    else:
       check_by_hand_2.append(account)

In [67]:
# USing regular expression to classify the greatest number of tweeter handles into governemnt accounts
political = []
check_by_hand_3 = []
pattern = re.compile(r'GOP|Gov|Senate|Senator|Rep', re.IGNORECASE) 
for item in check_by_hand_2:
    if pattern.search(item):
        political.append(item)
    else:
        check_by_hand_3.append(item)


In [68]:
political + ['RudyGiuliani', 'USDOT', 'ChineseEmbinHU', 'TeamCavuto', 'TheDemCoalition', 'Ohio_OCJS', 'SenSchumer', 'FEMAregion3', 'EPAregion3', 'FDA_ORA']
activists = ['PghProtests', 'CleanAirMoms', 'POGOwatchdog', 'genzforchange', 'PatrioticMills', 'TheUSASingers', 'OhioFarmBureau', 'gofundme', 'RevJJackson', 'AAF']
other = ['RedCross', 'scifri', 'AmChemistry', 'RedCrossNOH', 'GoyaFoods']

In [69]:
news_orgs_list3 = [i for i in check_by_hand_3 if i not in (political + activists + other)]

In [70]:
news_org = news_org_list + news_orgs_list2 + news_orgs_list3
news_org = list(set(news_org))

In [71]:
# Now, from all other accounts (non organizational , verified):

In [72]:
individual_verified = [i for i in data_3['Author'][(data_3['Account Type'] == 'individual') & (data_3['Twitter Verified'] == True)].value_counts().index]
individual_verified = [i for i in individual_verified if i not in (news_org + political + activists + other)]

In [73]:
import re
count = 0
news_ind_list = []
check_by_hand_ind = []
pattern = re.compile(r'news|abc|cnn|cbs|reuters|AP|times|fox|post|nbc|cnbc|bloomberg|chron|gazette|hours|politico|cspan|politics|msn|journal|Tribune|daily|wsj|yahoo|tv|sun|time|radio|chronicle|herald|today', re.IGNORECASE) 
for item in individual_verified:
    if pattern.search(item):
        news_ind_list.append(item)
    else:
        check_by_hand_ind.append(item)

In [74]:
news = news_org + news_ind_list

In [75]:
political_ind = []
check_by_hand_ind2 = []
pattern = re.compile(r'GOP|Gov|Senate|Senator|Rep|Sen|EPA', re.IGNORECASE) 
for item in check_by_hand_ind:
    if pattern.search(item):
        political_ind.append(item)
    else:
        check_by_hand_ind2.append(item)

In [76]:
politics = political + political_ind

In [ ]:
# Here, reading the cleaned data from source 2
MK_byhand = pd.read_csv('raw_data/check_by_hand_cleanMK.csv')
MK_byhand.set_index('check_by_hand_ind2', inplace=True)

In [78]:
MK_byhand.fillna(0, inplace=True)
news_org_additional = MK_byhand.query('news == 1').index.to_list()
government_additional = MK_byhand.query('political == 1').index.to_list()

In [79]:
check_by_hand_ind3 = [i for i in check_by_hand_ind2 if i not in (news_org_additional+government_additional)]

In [80]:
people = ['RobertKennedyJr', '_SemaHernandez_', 'CatoInstitute', 'ColMorrisDavis', 'mrtgr', 'LindseyBoylan', 'AJPennyfarthing', 'DrEricDing', 'mattkbh', 'AlBernstein', 'lootpress', 'DinahVP', 'iskandrah'\
          '_Zeets', 'vickorano', 'addedvalueth', 'IBDinvestors', 'SierraClub', 'StephenJ_Caruso', 'djrelentt', 'ErickFernandez', 'ceciliakcecilia', 'michaelmalice', 'court_bennett11'\
            'MsAConner', 'BrookingsEcon', 'MeredithLClark', 'drdavidmichaels', 'AWCities', 'laurenpeikoff', 'WajahatAli', 'MarkRuffalo', 'NewEconomics', 'EricMGarcia', 'GooRee']
political = ['TPPatriots', 'PalimenoForGAD1', 'ForTexasHoujami', 'ProgressOhio', 'mh4oh', 'ElectMattDolan', 'Mike_Pence', 'Heritage', 'NorthCarolinaGP', 'HCEMA', 'MaxMillerOH', 'DaveYostOH', 'Miller_Congress'\
             'NewDemCoalition', 'DebbieLesko', 'ElissaSlotkin', 'PADEPSecretary', 'BriannaForCO', 'MikeStuartWV', 'justin4all2', 'voteSmitherman', 'CongressmanRaja', 'jessicalbenham'\
              'AlexPadilla4CA', 'RedState', 'DemSocialists', 'JaredEMoskowitz']
journalists = [i for i in check_by_hand_ind3 if i not in (people + political)]

In [81]:
news_final = news + news_org_additional + journalists
government_final = politics + government_additional + political
people_final = people + activists + other

In [82]:

len(news_final), len(government_final), len(people_final)

(1821, 119, 48)

## parsing dataset 2

In [83]:
tweet = pd.read_csv('raw_data/Tweet.csv')
agent = pd.read_csv('raw_data/Agent.csv')
hashtag = pd.read_csv('raw_data/Hashtag.csv')
url = pd.read_csv('raw_data/Url.csv')
agent.rename(columns = {'Node Label' : 'Sender'}, inplace=True)
tweet_agent = pd.merge(tweet, agent, on=['Sender'])
tweet_agent[['Date', 'Time']] = tweet_agent['DATE'].str.split(' ', 1, expand=True)
tweet_agent.drop(columns=['Language_y','Location','DATE'], inplace = True)
tweet_agent.rename(columns={'Node ID_x' : 'TWEET_ID', 
                            'Node ID_y' : 'AGENT_ID', 
                            'Language_x' : 'LANGUAGE',
                            'conversation_id' : 'CONVERSATION_ID',
                            'Latitude' : 'LATITUDE',
                            'Longitude' : 'LONGITUDE',
                            'Sender' : 'SENDER',
                            'Date' : 'DATE',
                            'Time' : 'TIME'
                            }, inplace=True)
tweet_agent['IS_RETWEET'] = tweet_agent['IS_RETWEET'].fillna(0)
tweet_agent['HAS_URL'] = tweet_agent['HAS_URL'].fillna(0)
tweet_agent['IS_REPLY'] = tweet_agent['IS_REPLY'].fillna(0)
tweet_agent['IS_IN_REPLY_TO'] = tweet_agent['IS_IN_REPLY_TO'].fillna(0)
tweet_agent['IS_QUOTE'] = tweet_agent['IS_QUOTE'].fillna(0)
tweet_agent['IS_GEOTAGGED'] = tweet_agent['IS_GEOTAGGED'].fillna(0)
tweet_agent['IS_NEWS_AGENCY'] = tweet_agent['IS_NEWS_AGENCY'].fillna(0)
tweets_data = tweet_agent[tweet_agent.columns.difference(['CONVERSATION_ID','IS_GEOTAGGED','LATITUDE','LONGITUDE','AGENT_ID'])]

/var/folders/yv/9jkc42cj01gb_08dbbq9fcnw0000gn/T/ipykernel_96147/1648274890.py:7: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  tweet_agent[['Date', 'Time']] = tweet_agent['DATE'].str.split(' ', 1, expand=True)


In [84]:
data_3['HAS_URL'] = data_3['Expanded URLs'].apply(lambda x: len(x.split()) if not isinstance(x, float) else 0)
data_3['IS_REPLY'] = data_3['Engagement Type'].apply(lambda x : 1 if x == 'REPLY' else 0)
data_3['IS_QUOTE'] = data_3['Engagement Type'].apply(lambda x : 1 if x == 'QUOTE' else 0)
data_3['IS_RETWEET'] = data_3['Engagement Type'].apply(lambda x : 1 if x == 'RETWEET' else 0)
data_3['Twitter Verified'] = data_3['Twitter Verified'].apply(lambda x: 1 if x == True else 0)
drop_from_dataset3 = ['Query Id', 'Query Name', 'Title', 'Url', 'Domain', 'Page Type', 'Continent Code', 'Continent',
       'Country', 'City Code', 'Added', 'City', 'Entity Info', 'Expanded URLs', 'Full Name', 'Location Name',
       'Media URLs', 'Original Url', 'Professions',
       'Resource Id', 'Short URLs', 'Thread Author', 'Thread Created Date',
       'Thread Entry Type', 'Thread Id', 'Twitter Author ID', 'Twitter Reply to', 'Twitter Retweet of', 'Updated',
       'Reach (new)', 'Region', 'Region Code', 'Weblog Title', 'Engagement Type']

In [85]:
# These are columns that we may need to bring back, as they are interesting
drop_to_merge = ['Account Type', 'Sentiment', 'Hashtags', 'Impact', 'Interest', 'Mentioned Authors','Twitter Following',	'Twitter Reply Count', 'Twitter Tweets', 'Gender']

In [86]:
data_3.drop(columns=drop_from_dataset3, inplace=True)
data_3.drop(columns=drop_to_merge, inplace=True)

In [87]:
import re
data_3['DATE'] = data_3['Date'].apply(lambda x: x.split()[0])
data_3['TIME'] = data_3['Date'].apply(lambda x: x.split()[1])
data_3['TIME'] = data_3['TIME'].apply(lambda x: re.findall(r'(.*)\.', x)[0])
tweets_data['TIME'] = tweets_data['TIME'].apply(lambda x: re.findall(r'(.*)-', x)[0])
data_3.drop(columns='Date', inplace=True)

/var/folders/yv/9jkc42cj01gb_08dbbq9fcnw0000gn/T/ipykernel_96147/3624984072.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_data['TIME'] = tweets_data['TIME'].apply(lambda x: re.findall(r'(.*)-', x)[0])


In [88]:
# Rename
data_3.rename(columns= {'Author' :  'SENDER', 'Country Code' : 'LOCATION', 'Language' : 'LANGUAGE', 'Full Text' : 'MESSAGE', \
 'Twitter Verified': 'IS_VERIFIED', 'Twitter Followers':'NUMBER_FOLLOWERS', 'Twitter Retweets':'RETWEET_COUNT'}, inplace=True)

In [89]:
tweets_data.drop(columns = 'TWEET_ID', inplace=True)
tweets_data.drop_duplicates(subset = ['MESSAGE'], inplace = True)

/var/folders/yv/9jkc42cj01gb_08dbbq9fcnw0000gn/T/ipykernel_96147/2771311703.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_data.drop(columns = 'TWEET_ID', inplace=True)
/var/folders/yv/9jkc42cj01gb_08dbbq9fcnw0000gn/T/ipykernel_96147/2771311703.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_data.drop_duplicates(subset = ['MESSAGE'], inplace = True)


In [90]:
data = pd.concat([data_3, tweets_data], axis=0)
data.reset_index(drop=True, inplace=True)

In [91]:
data.drop_duplicates(subset=['MESSAGE'], inplace=True)

In [92]:
# Now, bring in the sender classifiers
data['IS_MEDIA'] = data.SENDER.apply(lambda x : 1 if x in news_final else 0)
data['IS_GOVT'] = data.SENDER.apply(lambda x : 1 if x in government_final else 0)
data['IS_PEOPLE'] = data[['IS_GOVT', 'IS_MEDIA']].apply(lambda x : 1 if x[0] == 0 | x[1] == 0 else 0, axis = 1)
data.SENDER[data.IS_MEDIA == 1].count(),  data.SENDER[data.IS_GOVT == 1].count(), data['IS_PEOPLE'].sum()

(7406, 376, 77438)

In [93]:
data.head(4)

,LANGUAGE,LOCATION,SENDER,MESSAGE,Impressions,NUMBER_FOLLOWERS,RETWEET_COUNT,Twitter Likes,IS_VERIFIED,HAS_URL,...,IS_QUOTE,IS_RETWEET,DATE,TIME,IS_IN_REPLY_TO,IS_NEWS_AGENCY,TWEET_COUNT,IS_MEDIA,IS_GOVT,IS_PEOPLE
0,en,USA,TurnKyBlue,RT @TristanSnell Get rid of train safety rules...,4414.0,4414.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2023-03-14,21:56:12,NaN,NaN,NaN,0,0,1
1,en,NaN,jrbkjrbk,RT @realTuckFrumper Ohio Sues Norfolk Southern...,1011.0,1011.0,0.0,0.0,0.0,1.0,...,0.0,1.0,2023-03-14,21:56:03,NaN,NaN,NaN,0,0,1
2,en,NaN,camoxendale,RT @SenJeffMerkley We need accountability from...,990.0,990.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2023-03-14,21:55:12,NaN,NaN,NaN,0,0,1
3,en,NaN,strongfemchar,RT @nicksortor 🚨 #BREAKING: The State of Ohio ...,64.0,64.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2023-03-14,21:54:42,NaN,NaN,NaN,0,0,1


In [ ]:
# Export the final cleaned data set for both LDA and sentiment analysis
data.to_csv('data/data_tweet.csv', index=False)

In [97]:
data.query('IS_MEDIA == 1')['Twitter Likes'].value_counts()

0.0       1845
1.0        483
2.0        354
3.0        243
4.0        159
          ... 
781.0        1
8806.0       1
214.0        1
2222.0       1
1702.0       1
Name: Twitter Likes, Length: 406, dtype: int64